# Cleaning Up

This notebook demonstrates how to clean up all the resources created in this set of PoC notebooks.

In [ ]:
# Python Built-Ins:
import time
import traceback

# External Dependencies:
import boto3  # (AWS Python SDK)

%store -r

# If any of these variables aren't recovered by the %store, you'll need to find and set them for this cleanup
# to work:
print(f"Dataset Group: {dataset_group_arn}")
print(f"S3 Bucket: {bucket_name}")
print(f"IAM Role: {personalize_role_arn}")

# Connect to Personalize API:
personalize = boto3.client("personalize")

Please note some steps rely on previous resource types being fully deleted before they can proceed. If you receive an error, wait a minute or two and try again.

## Clean up Campaigns

⚠️ This section will DELETE **all campaigns within the dataset_group_arn you configured above**.

In [ ]:
solutions = personalize.list_solutions(
    datasetGroupArn=dataset_group_arn,
    # NOTE: You will need to take additional steps in the (very unlikely!) event you have more than 100
    # solutions in this dataset group:
    maxResults=100,
)["solutions"]

any_campaigns_deleted = False

for solution in solutions:
    solution_arn = solution["solutionArn"]
    campaigns = personalize.list_campaigns(solutionArn=solution_arn)["campaigns"]
    for campaign in campaigns:
        campaign_arn = campaign["campaignArn"]
        print(f"DELETING CAMPAIGN {campaign_arn}")
        personalize.delete_campaign(campaignArn=campaign_arn)
        any_campaigns_deleted = True
        time.sleep(.2)

if any_campaigns_deleted:
    print(f"WAITING 4 minutes for campaign deletions to propagate")
    time.sleep(60 * 4)
print("Done!")

## Clean up Event Trackers

⚠️ This section will DELETE **all event trackers within the dataset_group_arn you configured**.

In [ ]:
trackers = personalize.list_event_trackers(
    datasetGroupArn=dataset_group_arn,
    # NOTE: You will need to take additional steps in the (very unlikely!) event you have more than 100
    # trackers in this dataset group:
    maxResults=100,
)["eventTrackers"]

for tracker in trackers:
    tracker_arn = tracker["eventTrackerArn"]
    print(f"DELETING EVENT TRACKER {tracker_arn}")
    personalize.delete_event_tracker(eventTrackerArn=tracker_arn)
    time.sleep(0.2)

if len(trackers):
    print(f"WAITING 30s for tracker deletions to propagate")
    time.sleep(30)
print("Done!")

## Clean up Filters

⚠️ This section will DELETE **all filters within the dataset_group_arn you configured**.

In [ ]:
filters = personalize.list_filters(
    datasetGroupArn=dataset_group_arn,
    # NOTE: You will need to take additional steps in the (very unlikely!) event you have more than 100
    # filters in this dataset group:
    maxResults=100,
)["Filters"]

for f in filters:
    filter_arn = f["filterArn"]
    print(f"DELETING FILTER {filter_arn}")
    personalize.delete_filter(filterArn=filter_arn)
    time.sleep(0.2)

if len(trackers):
    print(f"WAITING 30s for filter deletions to propagate")
    time.sleep(30)
print("Done!")

## Clean up Solutions

⚠️ This section will DELETE **all solutions within the dataset_group_arn you configured**.

In [ ]:
solutions = personalize.list_solutions(
    datasetGroupArn=dataset_group_arn,
    # NOTE: You will need to take additional steps in the (very unlikely!) event you have more than 100
    # solutions in this dataset group:
    maxResults=100,
)["solutions"]

for solution in solutions:
    solution_arn = solution["solutionArn"]
    #solnvers = personalize.list_solution_versions(solutionArn=solution_arn)["solutionVersions"]
    print(f"DELETING SOLUTION {solution_arn}")
    personalize.delete_solution(solutionArn=solution_arn)
    time.sleep(0.2)

if len(solutions):
    print(f"WAITING 120s for solution deletions to propagate")
    time.sleep(120)
print("Done!")

## Clean up Datasets and Schemas

⚠️ This section will DELETE:

- **All datasets within the dataset_group_arn you configured**
- **Any schemas not also used by other dataset groups**

In [ ]:
datasets = personalize.list_datasets(datasetGroupArn=dataset_group_arn)["datasets"]

schema_arns = []

for dataset in datasets:
    dataset_arn = dataset["datasetArn"]
    schema_arns.append(personalize.describe_dataset(datasetArn=dataset_arn)["dataset"]["schemaArn"])
    print(f"DELETING DATASET {dataset_arn}")
    personalize.delete_dataset(datasetArn=dataset_arn)
    time.sleep(0.2)

if len(datasets):
    print(f"WAITING 60s for dataset deletions to propagate")
    time.sleep(60)
print("Done!")

In [ ]:
any_schemas_deleted = False

for schema_arn in schema_arns:
    try:
        print(f"DELETING SCHEMA {schema_arn}")
        personalize.delete_schema(schemaArn=schema_arn)
        any_schemas_deleted = True
        time.sleep(0.2)
    except personalize.exceptions.ResourceNotFoundException:
        print(f"(already deleted)")
    except personalize.exceptions.ResourceInUseException as e:
        print(f"###### WARNING - Failed to delete schema - appears to be in-use by another dataset")
        traceback.print_exc()

if any_schemas_deleted:
    print(f"WAITING 60s for schema deletions to propagate")
    time.sleep(60)
print("Done!")

## Clean up the Dataset Group

Finally, clean up the empty dataset group:

In [ ]:
personalize.delete_dataset_group(datasetGroupArn=dataset_group_arn)

## Clean up the S3 bucket and IAM role

Start by deleting the role, then empty the bucket, then delete the bucket.

In [ ]:
iam = boto3.client("iam")

Identify the name of the role you want to delete.

You cannot delete an IAM role which still has policies attached to it. So after you have identified the relevant role, let's list the attached policies of that role.

In [ ]:
role_name = personalize_role_arn.partition("/")[2]

iam.list_attached_role_policies(RoleName=role_name)

You need to detach the policies in the result above using the code below. Repeat for each attached policy.

In [ ]:
iam.detach_role_policy(
    RoleName=role_name,
    PolicyArn="arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess",
)

In [ ]:
iam.detach_role_policy(
    RoleName=role_name,
    PolicyArn="arn:aws:iam::aws:policy/AmazonS3FullAccess",
)

Finally, you should be able to delete the IAM role.

In [ ]:
iam.delete_role(RoleName=role_name)

To delete an S3 bucket, it first needs to be empty. The safest way to delete an S3 bucket, is just to navigate to S3 in the AWS console, delete the objects in the bucket, and then delete the S3 bucket itself... The code below will also work, but be **very careful** about what buckets you point it at!!!

In [ ]:
# print(f"DELETING s3://{bucket_name}")
# time.sleep(5)
# !aws s3 rm --recursive s3://$bucket_name

# boto3.resource("s3").Bucket(bucket_name).delete()

## All done!

Thanks for following along! For more resources on Amazon Personalize, check out the [official samples repository](https://github.com/aws-samples/amazon-personalize-samples)